In [2]:
import pandas as pd
import os
import zipfile
import numpy as np
import cv2
import pickle

In [3]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity

In [13]:

models_path = "models/features.pkl"

In [14]:
# Paths to ZIP files
zip_files = [
    "/mnt/data/DeepFashion-MultiModal-main.zip",
    "/mnt/data/densepose-001.zip",
    "/mnt/data/images-002.zip"
]

# Extract each ZIP file
dataset_path = "/mnt/data/Fashion_Dataset/"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, "r") as zip_ref:
            zip_ref.extractall(dataset_path)
        print(f"✅ Extracted: {zip_file}")


In [ ]:
if not os.path.exists(dataset_path):
    with zipfile.ZipFile(dataset_zip, "r") as zip_ref:
        zip_ref.extractall(dataset_path)
    print("✅ Dataset extracted successfully!")

# Ensure model directory exists
if not os.path.exists(models_path):
    os.makedirs(models_path)

# Load Pretrained Model
model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

# Get Image Paths
# Image folders after extraction
image_folders = [
    os.path.join(dataset_path, "DeepFashion-MultiModal/img"),
    os.path.join(dataset_path, "densepose-001"),
    os.path.join(dataset_path, "images-002")
]

# Collect all images
image_files = []
for folder in image_folders:
    if os.path.exists(folder):
        image_files += [os.path.join(folder, img) for img in os.listdir(folder) if img.endswith(".jpg")]

print(f"✅ Total images found: {len(image_files)}")


# Feature Extraction
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

# Extract and Save Features
feature_list = np.array([extract_features(img) for img in image_files])
with open(os.path.join(models_path, "features.pkl"), "wb") as f:
    pickle.dump((image_files, feature_list), f)
print("✅ Feature extraction completed!")

# Capture Image from Webcam
def capture_image():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Capture Fashion Image")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Capture Fashion Image", frame)
        k = cv2.waitKey(1)
        if k % 256 == 27:  # ESC to exit
            print("Closing camera...")
            break
        elif k % 256 == 32:  # SPACE to capture
            img_name = "captured_image.jpg"
            cv2.imwrite(img_name, frame)
            print(f"Image saved as {img_name}")
            break
    cam.release()
    cv2.destroyAllWindows()

# Find Similar Outfit
def recommend_fashion():
    with open(os.path.join(models_path, "features.pkl"), "rb") as f:
        image_files, feature_list = pickle.load(f)
    captured_features = extract_features("captured_image.jpg")
    similarities = cosine_similarity([captured_features], feature_list)
    best_match_index = np.argmax(similarities)
    best_match = image_files[best_match_index]
    print(f"Recommended Outfit: {best_match}")
    img_recommend = cv2.imread(best_match)
    cv2.imshow("Recommended Outfit", img_recommend)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Run the system
if __name__ == "__main__":
    capture_image()
    recommend_fashion()


In [11]:
import os
print(os.path.exists("/mnt/data/DeepFashion-MultiModal-main.zip"))


False
